###Exploratory Data Analysis with Pyspark and Spark SQL
Samuel Giorno - Maxime Boiral - Lou Brunneau



In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
catalog = 'taxi_eda_db'
schema = 'yellow_taxi_trips'
volume = 'data'
file_name_2019 = 'yellow_tripdata_2019-01.parquet'
file_name_2025 = 'yellow_tripdata_2025-01.parquet'
path_volume = f'/Volumes/{catalog}/{schema}/{volume}'
download_url_2019 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-01.parquet'
download_url_2025 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet'
zone_lookup_url = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'


In [0]:
# Create catalog/schema/volume
spark.sql(f'CREATE CATALOG IF NOT EXISTS {catalog}')
spark.sql(f'CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}')
spark.sql(f'CREATE VOLUME IF NOT EXISTS {catalog}.{schema}.{volume}')

DataFrame[]

In [0]:
# Get the data
dbutils.fs.cp(download_url_2019, f"{path_volume}/{file_name_2019}")

# Load trips
df_trips = spark.read.parquet(f"{path_volume}/{file_name_2019}", header=True, inferSchema=True)
print(f"Loaded {df_trips.count()} trips")
df_trips.printSchema()

Loaded 7696617 trips
root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)



##Part 1
This section can be completed either using pyspark commands or sql commands ( There will be a section after in which a self-chosen subset of the questions are re-answered using the language not used for the main section. i.e. if pyspark is chosen for the main lab, sql should be used to repeat some of the questions. )

- Add a column that creates a unique key to identify each record in order to answer questions about individual trips


In [0]:
df_trips = df_trips.withColumn("trip_id", F.monotonically_increasing_id())


- Which trip has the highest passanger count


In [0]:
df_trips.orderBy(F.desc("passenger_count")).select("trip_id", "passenger_count", "tpep_pickup_datetime").show(1)


+-------+---------------+--------------------+
|trip_id|passenger_count|tpep_pickup_datetime|
+-------+---------------+--------------------+
| 949956|            9.0| 2019-01-05 13:12:29|
+-------+---------------+--------------------+
only showing top 1 row


- What is the Average passanger count



In [0]:
df_trips.select(F.avg("passenger_count")).show()

+--------------------+
|avg(passenger_count)|
+--------------------+
|  1.5670317144945614|
+--------------------+



- Shortest/longest trip by distance? by time?


In [0]:
# Shortest and longest trips by distance
print("Trip distance stats:")
df_trips.filter(F.col("trip_distance") > 0).select(
    F.min("trip_distance"), 
    F.max("trip_distance")
).show()

# By time
df_trips = df_trips.withColumn(
    "trip_duration_minutes",
    (F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 60
)

print("Trip duration stats:")
df_trips.filter(F.col("trip_duration_minutes") > 0).select(
    F.min("trip_duration_minutes"),
    F.max("trip_duration_minutes")
).show()

Trip distance stats:
+------------------+------------------+
|min(trip_distance)|max(trip_distance)|
+------------------+------------------+
|              0.01|             831.8|
+------------------+------------------+

Trip duration stats:
+--------------------------+--------------------------+
|min(trip_duration_minutes)|max(trip_duration_minutes)|
+--------------------------+--------------------------+
|      0.016666666666666666|         43648.01666666667|
+--------------------------+--------------------------+



- highest/lowest faire amounts for a trip, what burough is associated with the each.


- busiest day/slowest single day


In [0]:
df_trips = df_trips.withColumn("pickup_date", F.to_date("tpep_pickup_datetime"))
daily_trips = df_trips.groupBy("pickup_date").count().orderBy("count")

print("Slowest day:")
daily_trips.show(1)
print("Busiest day:")
daily_trips.orderBy(F.desc("count")).show(1)

Slowest day:
+-----------+-----+
|pickup_date|count|
+-----------+-----+
| 2018-12-21|    1|
+-----------+-----+
only showing top 1 row
Busiest day:
+-----------+------+
|pickup_date| count|
+-----------+------+
| 2019-01-25|292499|
+-----------+------+
only showing top 1 row


- busiest/slowest time of day ( you may want to bucket these by hour or create timess such as morning, afternoon, evening, late night )


In [0]:
# Time of day analysis
df_trips = df_trips.withColumn("pickup_hour", F.hour("tpep_pickup_datetime"))

print("Trips by hour:")
df_trips.groupBy("pickup_hour").count().orderBy("pickup_hour").show(24)

# Time periods
df_trips = df_trips.withColumn(
    "time_period",
    F.when((F.col("pickup_hour") >= 6) & (F.col("pickup_hour") < 12), "Morning")
     .when((F.col("pickup_hour") >= 12) & (F.col("pickup_hour") < 17), "Afternoon")
     .when((F.col("pickup_hour") >= 17) & (F.col("pickup_hour") < 22), "Evening")
     .otherwise("Late Night")
)

df_trips.groupBy("time_period").count().orderBy(F.desc("count")).show()


Trips by hour:
+-----------+------+
|pickup_hour| count|
+-----------+------+
|          0|207842|
|          1|149254|
|          2|109421|
|          3| 78086|
|          4| 61424|
|          5| 75533|
|          6|178598|
|          7|304858|
|          8|373742|
|          9|365935|
|         10|361390|
|         11|375441|
|         12|401173|
|         13|404153|
|         14|433139|
|         15|452691|
|         16|420843|
|         17|468479|
|         18|515390|
|         19|475186|
|         20|423156|
|         21|409901|
|         22|369041|
|         23|281941|
+-----------+------+

+-----------+-------+
|time_period|  count|
+-----------+-------+
|    Evening|2292112|
|  Afternoon|2111999|
|    Morning|1959964|
| Late Night|1332542|
+-----------+-------+



- On average which day of the week is slowest/busiest


In [0]:
df_trips = df_trips.withColumn("day_of_week", F.dayofweek("tpep_pickup_datetime"))
df_trips = df_trips.withColumn(
    "day_name",
    F.when(F.col("day_of_week") == 1, "Sunday")
     .when(F.col("day_of_week") == 2, "Monday")
     .when(F.col("day_of_week") == 3, "Tuesday")
     .when(F.col("day_of_week") == 4, "Wednesday")
     .when(F.col("day_of_week") == 5, "Thursday")
     .when(F.col("day_of_week") == 6, "Friday")
     .otherwise("Saturday")
)

print("Trips by day of week:")
df_trips.groupBy("day_name", "day_of_week").count().orderBy("day_of_week").show()


Trips by day of week:
+---------+-----------+-------+
| day_name|day_of_week|  count|
+---------+-----------+-------+
|   Sunday|          1| 859905|
|   Monday|          2| 908121|
|  Tuesday|          3|1209084|
|Wednesday|          4|1265264|
| Thursday|          5|1357043|
|   Friday|          6|1087215|
| Saturday|          7|1009985|
+---------+-----------+-------+



- Does trip distance or num passangers affect tip amount


In [0]:
# Correlation analysis - distance/passengers vs tip
df_trips.filter(
    (F.col("tip_amount") >= 0) & (F.col("trip_distance") > 0)
).select(
    F.corr("trip_distance", "tip_amount").alias("distance_tip_corr"),
    F.corr("passenger_count", "tip_amount").alias("passenger_tip_corr")
).show()

# Tips by passenger count
df_trips.filter(
    (F.col("tip_amount") >= 0) & (F.col("passenger_count") > 0) & (F.col("passenger_count") <= 6)
).groupBy("passenger_count").agg(
    F.avg("tip_amount").alias("avg_tip"),
    F.count("*").alias("trips")
).orderBy("passenger_count").show()

+------------------+--------------------+
| distance_tip_corr|  passenger_tip_corr|
+------------------+--------------------+
|0.5471894202380073|8.781409231332617E-4|
+------------------+--------------------+

+---------------+------------------+-------+
|passenger_count|           avg_tip|  trips|
+---------------+------------------+-------+
|            1.0|1.8284086254890013|5456434|
|            2.0|1.8339754318026735|1113879|
|            3.0| 1.795606914741546| 314690|
|            4.0|1.7027265687165505| 140752|
|            5.0|1.8699009072436963| 323838|
|            6.0|1.8568570631795251| 200809|
+---------------+------------------+-------+



- What was the highest "extra" charge and which trip


In [0]:
df_trips.orderBy(F.desc("extra")).select("trip_id", "extra", "total_amount", "tpep_pickup_datetime").show(1)


+-------+------+------------+--------------------+
|trip_id| extra|total_amount|tpep_pickup_datetime|
+-------+------+------------+--------------------+
|5323483|535.38|   356214.78| 2019-01-23 08:58:09|
+-------+------+------------+--------------------+
only showing top 1 row


- Are there any datapoints that seem to be strange/outliers (make sure to explain your reasoning in a markdown cell)?

In [0]:
df_trips.select(
    F.sum(F.when(F.col("passenger_count") == 0, 1).otherwise(0)).alias("zero_passengers"),
    F.sum(F.when(F.col("trip_distance") == 0, 1).otherwise(0)).alias("zero_distance"),
    F.sum(F.when(F.col("fare_amount") < 0, 1).otherwise(0)).alias("negative_fare")
).show()

print("Outliers:")
print(f"Trips over 100 miles: {df_trips.filter(F.col('trip_distance') > 100).count()}")
print(f"Trips over 3 hours: {df_trips.filter(F.col('trip_duration_minutes') > 180).count()}")
print(f"Trips over $500: {df_trips.filter(F.col('total_amount') > 500).count()}")
print(f"Trips with >6 passengers: {df_trips.filter(F.col('passenger_count') > 6).count()}")


+---------------+-------------+-------------+
|zero_passengers|zero_distance|negative_fare|
+---------------+-------------+-------------+
|         117381|        55089|         7129|
+---------------+-------------+-------------+

Outliers:
Trips over 100 miles: 32
Trips over 3 hours: 20914
Trips over $500: 56
Trips with >6 passengers: 57


##Part 2
- Using the code for loading the first dataset as an example, load in the taxi zone lookup and answer the following questions


In [0]:
# Load zone lookup
zone_file = 'taxi_zone_lookup.csv'
dbutils.fs.cp(zone_lookup_url, f"{path_volume}/{zone_file}")
df_zones = spark.read.csv(f"{path_volume}/{zone_file}", header=True, inferSchema=True)

print("Zone lookup loaded:")
df_zones.show(10)

# Join with zones
df_trips_zones = df_trips \
    .join(df_zones.withColumnRenamed("LocationID", "PULocationID")
                  .withColumnRenamed("Borough", "pickup_borough")
                  .withColumnRenamed("Zone", "pickup_zone"),
          "PULocationID", "left") \
    .join(df_zones.withColumnRenamed("LocationID", "DOLocationID")
                  .withColumnRenamed("Borough", "dropoff_borough")
                  .withColumnRenamed("Zone", "dropoff_zone"),
          "DOLocationID", "left")

Zone lookup loaded:
+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 10 rows


- which borough had most pickups? dropoffs?


In [0]:
print("Pickups by borough:")
df_trips_zones.groupBy("pickup_borough").count().orderBy(F.desc("count")).show()

# Dropoffs by borough
print("Dropoffs by borough:")
df_trips_zones.groupBy("dropoff_borough").count().orderBy(F.desc("count")).show()




Pickups by borough:
+--------------+-------+
|pickup_borough|  count|
+--------------+-------+
|     Manhattan|6950965|
|        Queens| 471173|
|       Unknown| 159815|
|      Brooklyn|  91905|
|         Bronx|  18062|
|           N/A|   3890|
|           EWR|    446|
| Staten Island|    361|
+--------------+-------+

Dropoffs by borough:
+---------------+-------+
|dropoff_borough|  count|
+---------------+-------+
|      Manhattan|6817355|
|         Queens| 340972|
|       Brooklyn| 301105|
|        Unknown| 149097|
|          Bronx|  58085|
|            N/A|  16904|
|            EWR|  10914|
|  Staten Island|   2185|
+---------------+-------+



- what are the busiest days of the week by borough?


In [0]:
print("Busiest days by borough:")
borough_weekly = df_trips_zones.groupBy("pickup_borough", "day_name").count()
for borough in ["Manhattan", "Queens", "Brooklyn", "Bronx"]:
    print(f"\n{borough}:")
    borough_weekly.filter(F.col("pickup_borough") == borough).orderBy(F.desc("count")).show(3)


Busiest days by borough:

Manhattan:
+--------------+---------+-------+
|pickup_borough| day_name|  count|
+--------------+---------+-------+
|     Manhattan| Thursday|1229554|
|     Manhattan|Wednesday|1144782|
|     Manhattan|  Tuesday|1086202|
+--------------+---------+-------+
only showing top 3 rows

Queens:
+--------------+---------+-----+
|pickup_borough| day_name|count|
+--------------+---------+-----+
|        Queens| Thursday|78972|
|        Queens|  Tuesday|78684|
|        Queens|Wednesday|75831|
+--------------+---------+-----+
only showing top 3 rows

Brooklyn:
+--------------+---------+-----+
|pickup_borough| day_name|count|
+--------------+---------+-----+
|      Brooklyn|  Tuesday|15779|
|      Brooklyn| Thursday|15714|
|      Brooklyn|Wednesday|15101|
+--------------+---------+-----+
only showing top 3 rows

Bronx:
+--------------+---------+-----+
|pickup_borough| day_name|count|
+--------------+---------+-----+
|         Bronx| Thursday| 3121|
|         Bronx|  Tuesda

- what is the average trip distance by borough?


In [0]:
print("Average trip distance by borough:")
df_trips_zones.filter(F.col("trip_distance") > 0).groupBy("pickup_borough").agg(
    F.avg("trip_distance").alias("avg_distance"),
    F.count("*").alias("trips")
).orderBy(F.desc("avg_distance")).show()

Average trip distance by borough:
+--------------+------------------+-------+
|pickup_borough|      avg_distance|  trips|
+--------------+------------------+-------+
| Staten Island|13.761585365853659|    328|
|        Queens| 11.59852848391229| 458364|
|           EWR|7.6988888888888845|    153|
|         Bronx| 7.554846469669801|  17293|
|           N/A|  5.59139513951397|   2222|
|      Brooklyn| 4.905750504498675|  89693|
|       Unknown|2.5447266590637896| 151697|
|     Manhattan| 2.238066946093927|6921778|
+--------------+------------------+-------+



- what is the average trip fare by borough?


In [0]:
print("Average fare by borough:")
df_trips_zones.filter(F.col("total_amount") > 0).groupBy("pickup_borough").agg(
    F.avg("total_amount").alias("avg_fare"),
    F.count("*").alias("trips")
).orderBy(F.desc("avg_fare")).show()

Average fare by borough:
+--------------+------------------+-------+
|pickup_borough|          avg_fare|  trips|
+--------------+------------------+-------+
|           EWR|  94.5367573696146|    441|
|           N/A| 71.13499739039354|   3832|
| Staten Island| 54.59456338028171|    355|
|        Queens| 44.66860808294255| 469209|
|         Bronx|29.596831280308137|  17941|
|      Brooklyn| 21.72761210492711|  91566|
|       Unknown|18.266325743523833| 159205|
|     Manhattan|13.685106296390218|6945298|
+--------------+------------------+-------+



- load the dataset from the most recently available january, is there a change to any of the average metrics.

In [0]:
try:
    dbutils.fs.cp(download_url_2025, f"{path_volume}/{file_name_2025}")
    df_trips_2025 = spark.read.parquet(f"{path_volume}/{file_name_2025}", header=True, inferSchema=True)
    
    df_trips_2025 = df_trips_2025.withColumn(
        "trip_duration_minutes",
        (F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 60
    )
    
    print(f"2025 data loaded: {df_trips_2025.count()} trips")
    
    # Compare 2019 vs 2025
    print("\n2019 metrics:")
    df_trips.select(
        F.count("*").alias("trips"),
        F.avg("trip_distance").alias("avg_distance"),
        F.avg("total_amount").alias("avg_fare"),
        F.avg("passenger_count").alias("avg_passengers")
    ).show()
    
    print("2025 metrics:")
    df_trips_2025.select(
        F.count("*").alias("trips"),
        F.avg("trip_distance").alias("avg_distance"),
        F.avg("total_amount").alias("avg_fare"),
        F.avg("passenger_count").alias("avg_passengers")
    ).show()
    
except Exception as e:
    print(f"Couldn't load 2025 data: {e}")

2025 data loaded: 3475226 trips

2019 metrics:
+-------+------------------+------------------+------------------+
|  trips|      avg_distance|          avg_fare|    avg_passengers|
+-------+------------------+------------------+------------------+
|7696617|2.8301461681151574|15.810651344610736|1.5670317144945614|
+-------+------------------+------------------+------------------+

2025 metrics:
+-------+-----------------+-----------------+------------------+
|  trips|     avg_distance|         avg_fare|    avg_passengers|
+-------+-----------------+-----------------+------------------+
|3475226|5.855126178844192|25.61129169728817|1.2978589658806226|
+-------+-----------------+-----------------+------------------+




##Part 3


- view for SQL 

In [0]:
df_trips.createOrReplaceTempView("trips")
df_zones.createOrReplaceTempView("zones")

- Average passenger count in SQL

In [0]:
spark.sql("""
    SELECT 
        AVG(passenger_count) as avg_passengers,
        MIN(passenger_count) as min_passengers,
        MAX(passenger_count) as max_passengers
    FROM trips
    WHERE passenger_count > 0
""").show()

+-----------------+--------------+--------------+
|   avg_passengers|min_passengers|max_passengers|
+-----------------+--------------+--------------+
|1.591392775427107|           1.0|           9.0|
+-----------------+--------------+--------------+



- Busiest day of week in SQL

In [0]:
spark.sql("""
    SELECT 
        CASE dayofweek(tpep_pickup_datetime)
            WHEN 1 THEN 'Sunday'
            WHEN 2 THEN 'Monday'
            WHEN 3 THEN 'Tuesday'
            WHEN 4 THEN 'Wednesday'
            WHEN 5 THEN 'Thursday'
            WHEN 6 THEN 'Friday'
            ELSE 'Saturday'
        END as day_name,
        COUNT(*) as trips
    FROM trips
    GROUP BY dayofweek(tpep_pickup_datetime)
    ORDER BY trips DESC
""").show()

+---------+-------+
| day_name|  trips|
+---------+-------+
| Thursday|1357043|
|Wednesday|1265264|
|  Tuesday|1209084|
|   Friday|1087215|
| Saturday|1009985|
|   Monday| 908121|
|   Sunday| 859905|
+---------+-------+



- Top routes by borough with JOIN

In [0]:
spark.sql("""
    SELECT 
        pickup.Borough as pickup_borough,
        dropoff.Borough as dropoff_borough,
        COUNT(*) as trips,
        AVG(t.trip_distance) as avg_distance,
        AVG(t.total_amount) as avg_fare
    FROM trips t
    LEFT JOIN zones pickup ON t.PULocationID = pickup.LocationID
    LEFT JOIN zones dropoff ON t.DOLocationID = dropoff.LocationID
    WHERE pickup.Borough IS NOT NULL 
      AND dropoff.Borough IS NOT NULL
    GROUP BY pickup.Borough, dropoff.Borough
    ORDER BY trips DESC
    LIMIT 10
""").show()

+--------------+---------------+-------+------------------+------------------+
|pickup_borough|dropoff_borough|  trips|      avg_distance|          avg_fare|
+--------------+---------------+-------+------------------+------------------+
|     Manhattan|      Manhattan|6504842|1.7786051359893669|12.011004899989434|
|        Queens|      Manhattan| 260406|13.175084483460472|52.333278419078134|
|     Manhattan|         Queens| 203936| 10.18861520280864| 41.33202651804795|
|     Manhattan|       Brooklyn| 177506|6.7671669126677445|29.436141482504564|
|       Unknown|        Unknown| 138613| 2.488677468924241| 17.53346915513194|
|        Queens|         Queens| 126535| 5.239023353222512|22.869027857889517|
|        Queens|       Brooklyn|  65488|13.554985493525438|  46.6634278035518|
|      Brooklyn|       Brooklyn|  55622| 2.577569307108713|14.453794361946862|
|     Manhattan|          Bronx|  39076| 9.175761848704967|   33.015027382548|
|      Brooklyn|      Manhattan|  27657| 7.030075930

##Part 4

In [0]:
# Using Spark's native display
print("Hourly traffic:")
hourly_viz = df_trips.groupBy("pickup_hour").count().orderBy("pickup_hour")
display(hourly_viz)

print("Weekly traffic:")
weekly_viz = df_trips.groupBy("day_name", "day_of_week").count().orderBy("day_of_week")
display(weekly_viz)

print("Fare by borough:")
fare_viz = df_trips_zones.filter(F.col("total_amount") > 0).groupBy("pickup_borough").agg(
    F.avg("total_amount").alias("avg_fare")
).orderBy(F.desc("avg_fare"))
display(fare_viz)

Hourly traffic:


pickup_hour,count
0,207842
1,149254
2,109421
3,78086
4,61424
5,75533
6,178598
7,304858
8,373742
9,365935


Weekly traffic:


day_name,day_of_week,count
Sunday,1,859905
Monday,2,908121
Tuesday,3,1209084
Wednesday,4,1265264
Thursday,5,1357043
Friday,6,1087215
Saturday,7,1009985


Fare by borough:


pickup_borough,avg_fare
EWR,94.5367573696146
N/A,71.13499739039354
Staten Island,54.59456338028171
Queens,44.66860808294255
Bronx,29.596831280308137
Brooklyn,21.72761210492711
Unknown,18.266325743523833
Manhattan,13.685106296390218
